In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
import pandas as pd
df = pd.read_json(r"/home/jovyan/thesis/vec.json")

In [3]:
X = df['RNN vectors'].to_list() 
y = df['Encoded Categories'].to_list() 

max_len = max(len(x) for x in X)# Fix Shape
X = np.array([np.concatenate([x, [0] * (max_len - len(x))]) for x in X])

features = X
target = y

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred_rnn = rf.predict(X_test)
f1score_rnn = f1_score(y_test, y_pred_rnn, average='weighted')
print(f"F1-score: {f1score_rnn}")

F1-score: 0.053190714589171274


In [4]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score

In [8]:
# Calculate confusion matrix
cm_rnn = confusion_matrix(y_test, y_pred_rnn)
print("Confusion matrix:")
print(cm_rnn)

Confusion matrix:
[[334   0   0 ...   0   0   0]
 [  3   0   0 ...   0   0   0]
 [  2   0   0 ...   0   0   0]
 ...
 [  2   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]]


In [10]:
# Calculate accuracy
acc_rnn = accuracy_score(y_test, y_pred_rnn)
print(f"Accuracy: {acc_rnn}")

Accuracy: 0.16133584195672623


In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
print("Classification report:")
print(classification_report(y_test, y_pred_rnn))

Classification report:
              precision    recall  f1-score   support

           0       0.17      0.96      0.29       349
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         5
          29       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         1
    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
